<a href="https://colab.research.google.com/github/shahmancodes/Course-Database/blob/main/Course_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import spacy
import pandas as pd

In [3]:
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
ust = pd.read_excel("HKUST Comp.xlsx").dropna()
hku = pd.read_csv("HKU Comp.csv")

In [5]:
'''
w1 = ust["Description"][0]
w2 = ust["Title"][0]
w1 = nlp(w1)
w2 = nlp(w2)
w2s = [sent.text for sent in w2.sents]
w2s
for i in range(len(hku)):
  y = nlp(hku["Description"][i])
  w3ns = nlp(' '.join([str(t) for t in y if not t.is_stop]))
  q = w1ns.similarity(w3ns)
  print(q)

'''

'\nw1 = ust["Description"][0]\nw2 = ust["Title"][0]\nw1 = nlp(w1)\nw2 = nlp(w2)\nw2s = [sent.text for sent in w2.sents]\nw2s\nfor i in range(len(hku)):\n  y = nlp(hku["Description"][i])\n  w3ns = nlp(\' \'.join([str(t) for t in y if not t.is_stop]))\n  q = w1ns.similarity(w3ns)\n  print(q)\n\n'

In [6]:
!pip install spacy-universal-sentence-encoder
import spacy_universal_sentence_encoder
# load one of the models: ['en_use_md', 'en_use_lg', 'xx_use_md', 'xx_use_lg']
nlp = spacy_universal_sentence_encoder.load_model('en_use_lg')

  Preparing metadata (setup.py) ... done
  Created wheel for spacy-universal-sentence-encoder: filename=spacy_universal_sentence_encoder-0.4.6-py3-none-any.whl size=16539 sha256=d6047490b7b1b273dba6725bc8ce4e91c3e5804779282712d6f238699813b0f3
  Stored in directory: /root/.cache/pip/wheels/65/75/56/7fd780bb5e1f9c74625ccda3c03a1abeb27a2ade5a8946db33
Successfully built spacy-universal-sentence-encoder
Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB



In [7]:
courses = {0:[0,1],1:[0],2:[2],3:[0],4:[0,1],5:[],6:[17],7:[1],8:[9],9:[9],10:[26],11:[7],12:[38],13:[8],14:[8],15:[],16:[],17:[25],18:[0],19:[16],20:[16],21:[]}

In [8]:
w1 = ust["Description"][13]
w2 = ust["Title"][13]
w1 = nlp(w1)
w2 = nlp(w2)
w3 = hku["Title"][8]
w3 = nlp(w3)

In [9]:
'''for i in range(len(hku)):
  y = nlp(hku["Title"][i])
  q = w2.similarity(y)
  print(i,q)
'''
correct = incorrect = max_score = mappingNA = 0
documents = [hku["Description"][i] for i in range(len(hku))] #A list with all the Descriptions of HKU Courses
for i in range(len(ust)):
  documents.insert(0,ust["Description"][i]) #Appending  UST Description to the same list, at Index 0, one by one
  w1 = nlp(ust["Description"][i])
  for j in range(len(hku)):
    w2 = nlp(hku["Description"][j])
    sim_score = w1.similarity(w2)
    if sim_score > max_score:
        max_score = sim_score
        best_sim_index = j-1

  found= 2
  if i in courses:

    for numberofmapping in range(len(courses[i])):
      if courses[i][numberofmapping] == best_sim_index:
        found=1
      else:
        found=0

    if (found==1):
      correct=correct+1
    if (found==0):
      incorrect=incorrect+1
    if (found==2):
      mappingNA=mappingNA+1

  del documents[0]
accuracy = correct/(correct+incorrect)
accuracy,correct,incorrect,mappingNA


(0.16666666666666666, 3, 15, 4)

In [10]:
#Sklearn

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
'''
# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Print the similarity matrix
print("Cosine similarity matrix:")
all_course = np.delete(cosine_sim[0],0)

best_sim_index = all_course.argmax()'''
correct = incorrect = mappingNA = 0
documents = [hku["Description"][i] for i in range(len(hku))]
for i in range(len(ust)):
  documents.insert(0,ust["Description"][i])
  # Create a TF-IDF vectorizer
  vectorizer = TfidfVectorizer()

  # Fit and transform the documents
  tfidf_matrix = vectorizer.fit_transform(documents)

  # Compute cosine similarity
  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

  all_course = np.delete(cosine_sim[0],0)

  best_sim_index = all_course.argmax()

  found= 2
  if i in courses:

    for numberofmapping in range(len(courses[i])):
      if courses[i][numberofmapping] == best_sim_index:
        found=1
      else:
        found=0

    if (found==1):
      correct=correct+1
    if (found==0):
      incorrect=incorrect+1
    if (found==2):
      mappingNA=mappingNA+1

  del documents[0]
accuracy = correct/(correct+incorrect)
accuracy,correct,incorrect,mappingNA

(0.3333333333333333, 6, 12, 4)

In [12]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def compute_similarity(hku, ust, courses):
    correct = incorrect = mappingNA = 0
    documents = [hku["Description"][i] for i in range(len(hku))]

    for i in range(len(ust)):
        documents.insert(0, ust["Description"][i])

        # Create a TF-IDF vectorizer
        vectorizer = TfidfVectorizer()

        # Fit and transform the documents
        tfidf_matrix = vectorizer.fit_transform(documents)

        # Compute cosine similarity
        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

        all_course = np.delete(cosine_sim[0], 0)

        best_sim_index = all_course.argmax()

        found = 2
        if i in courses:
            for numberofmapping in range(len(courses[i])):
                if courses[i][numberofmapping] == best_sim_index:
                    found = 1
                else:
                    found = 0

            if found == 1:
                correct += 1
            elif found == 0:
                incorrect += 1
            elif found == 2:
                mappingNA += 1

        del documents[0]

    accuracy = correct / (correct + incorrect)
    return accuracy, correct, incorrect, mappingNA
'''

'\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.metrics.pairwise import cosine_similarity\nimport numpy as np\n\ndef compute_similarity(hku, ust, courses):\n    correct = incorrect = mappingNA = 0\n    documents = [hku["Description"][i] for i in range(len(hku))]\n\n    for i in range(len(ust)):\n        documents.insert(0, ust["Description"][i])\n\n        # Create a TF-IDF vectorizer\n        vectorizer = TfidfVectorizer()\n\n        # Fit and transform the documents\n        tfidf_matrix = vectorizer.fit_transform(documents)\n\n        # Compute cosine similarity\n        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)\n\n        all_course = np.delete(cosine_sim[0], 0)\n\n        best_sim_index = all_course.argmax()\n\n        found = 2\n        if i in courses:\n            for numberofmapping in range(len(courses[i])):\n                if courses[i][numberofmapping] == best_sim_index:\n                    found = 1\n                else:\n  

In [13]:
compute_similarity(hku, ust, courses)

NameError: name 'compute_similarity' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#def compute_similarity(hku, ust, courses):
correct = incorrect = mappingNA = user_input = 0
asked_courses = {}
documents = [hku["Description"][i] for i in range(len(hku))]
on_off = np.zeros(len(hku))
while user_input != "stop":
  user_input = input("Enter course code")
  if user_input == "stop" or user_input not in ust["Code"].values:
    break
  if user_input not in asked_courses:

    # inserts input course description to documents list
    documents.insert(0, ust.loc[ust['Code'] == user_input, 'Description'].iloc[0])

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the documents
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Compute cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    #deletes input course from documents list
    best_sim_score = np.delete(cosine_sim[0], 0)
    asked_courses[user_input] = best_sim_score

  #Finding the index of the most similar course
  best_sim_score_index = best_sim_score[0].argmax()
  print(hku.loc[best_sim_score_index, "Code"])

  #Asking for user feedback and updating the model accordingly
  user_input_2 = input("Is the course matching correct?")
  if user_input_2 == "yes":
    print(type(best_sim_score))
    #We can increase the similarity score of matched course to strengthen the program prediction
    print("hi")
  elif user_input_2 == "no":
    #Updating the model slowly for better predictions
    #Adding ones to incorrectly predicted course

    on_off[best_sim_score_index] = 1
    #Appending array to similarity array
    if best_sim_score.shape[0]==2:
      best_sim_score = np.array([best_sim_score[0],on_off])
    else:
      best_sim_score = np.array([best_sim_score,on_off])

    # Find the indices where the value in the second row is 1
    indices = np.where(best_sim_score[1] == 0)[0]
    print(best_sim_score,best_sim_score_index)
    # Multiply the values in the first row by 2 at the corresponding indices
    best_sim_score[0, indices] *= 1.1
'''
    best_sim_score = np.array([besgt_sim_score,on_off])
    best_sim_score[0]*1.1 if best_sim_score[]
    best_sim_score[best_sim_score != best_sim_score[best_sim_score_index]] *= 1.10
'''


asked_courses
hku.loc[best_sim_score_index, "Code"]
      #globals()[user_input+"sim_score"] = np.delete(cosine_sim[0], 0)

      #globals()[user_input+"sim_score"].argmax()


Enter course codeCOMP 1021
COMP1117
Is the course matching correct?no
[[0.19310837 0.18952149 0.22301935 0.11400961 0.21478104 0.22301935
  0.01405622 0.16426892 0.0884518  0.12995618 0.09887518 0.01710477
  0.03877105 0.24982923 0.16047723 0.0691727  0.12870533 0.21695475
  0.14145953 0.10874222 0.12360356 0.15528918 0.21478104 0.14455283
  0.24548355 0.14034335 0.04753333 0.07529198 0.10582215 0.15180481
  0.09386971 0.24990803 0.12648983 0.29723635 0.18639266 0.0765238
  0.         0.15266872 0.08273984 0.20454527 0.11564591 0.07634939
  0.01062401 0.1699267  0.1942388  0.15789675]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.

In [54]:
import numpy as np
hi1 = np.zeros((4,2))
print(hi1.shape)


(4, 2)


In [36]:
data = np.array([[2, 3, 4, 5],
                 [1, 0, 1, 1]])

# Find the indices where the value in the second row is 1
indices = np.where(data[1] == 1)[0]

# Multiply the values in the first row by 2 at the corresponding indices
data[0, indices] *= 2

indices

array([0, 2, 3])

In [ ]:
feedback = {}
while user_choice != "stop":
  user_choice = input("Enter course code")
  user_input = int(input("Do you think this is the correct course matching? 1 for yes and 0 for no "))
  if user_choice not in feedback:
    feedback[user_choice] = []
  feedback[user_choice].append(user_input)

In [ ]:
feedback = {}
user_choice = ""
while user_choice != "stop":
  user_choice = input("Enter course code")
  user_input = int(input("Do you think this is the correct course matching? 1 for yes and 0 for no "))
  if user_choice not in feedback:
    feedback[user_choice] = []
  feedback[user_choice].append(user_input)
